In [ ]:
import bittensor as bt

bt.logging.set_debug(True)
tensor = bt.subtensor("archive")

start_block_number = 2978959
blocks_to_search = 1000

total_subnets = 34
subnets_range = range(1, total_subnets + 1)

# Collect current axon version

In [ ]:
class AxonInfo:
    def __init__(self, hotkey: str, version: int):
        self.hotkey = hotkey
        self.version = version

    def __repr__(self):
        return f"AxonInfo(hotkey={self.hotkey}, version={self.version})"

def get_axon_infos_for_subnet(net_id: int) -> list[AxonInfo]:
    metagraph = tensor.metagraph(net_id, block=start_block_number)

    uids_list = [id for id in metagraph.uids.tolist() if metagraph.validator_permit[id]]
    neurons = [tensor.neuron_for_uid(uid, net_id, block=start_block_number) for uid in uids_list]
    return [AxonInfo(neuron.hotkey, neuron.prometheus_info.version) for neuron in neurons]

def get_axon_infos_for_all_subnets() -> list[list[AxonInfo]]:
    return [get_axon_infos_for_subnet(net_id) for net_id in subnets_range]

axon_infos = get_axon_infos_for_all_subnets()
axon_infos

# Collect latest submitted version

In [ ]:
from dataclasses import dataclass
from scalecodec import GenericExtrinsic

@dataclass
class Extrinsic:
    txHash: str
    signer: str
    success: bool
    subnet: int
    version: int

substrate = tensor.substrate

def get_set_weight_extrinsics(block_number: int) -> list[Extrinsic]:
    block = substrate.get_block(block_number=block_number)
    events = substrate.get_events(block['header']['hash'])

    failed_extrinsics = []

    for event in events:
        event_data = event.value['event']
        if event_data['event_id'] == 'ExtrinsicFailed':
            failed_extrinsics.append(event.value['extrinsic_idx'])

    extrinsics = block['extrinsics']
    result = []

    for index, extrinsic in enumerate(extrinsics):
        extrinsic = extrinsic.value
        
        call = extrinsic['call']
        if call['call_function'] == 'set_weights':
            extrinsic_hash = extrinsic['extrinsic_hash']
            signer = extrinsic['address']
            success = failed_extrinsics.count(index) == 0
            
            call_args = call['call_args']
            subnet = call_args[0]['value']
            version = call_args[-1]['value']
            result.append(Extrinsic(extrinsic_hash, signer, success, subnet, version))

    return result
    
def get_set_weight_extrinsics_all(starting_block_number: int, blocks_to_search: int) -> list[Extrinsic]:
    result = []
    for i in range(blocks_to_search):
        block_number = starting_block_number - i
        try:
            result += get_set_weight_extrinsics(block_number)
        except Exception as e:
            print(f"Error at block {block_number}: {e}")
    return result

all_extrinsics = get_set_weight_extrinsics_all(start_block_number, blocks_to_search)
all_extrinsics

In [ ]:
# Subnet ID -> Hotkey -> First non faild extrinsic
aggregated_extrinsics = {}

for extrinsic in all_extrinsics:
    if extrinsic.success:
        subnet = extrinsic.subnet
        hotkey = extrinsic.signer
        success = extrinsic.success

        if not success:
            continue
        
        if subnet not in aggregated_extrinsics:
            aggregated_extrinsics[subnet] = {}

        if hotkey not in aggregated_extrinsics[subnet]:
            aggregated_extrinsics[subnet][hotkey] = extrinsic

# Merge data

In [ ]:
# Subnet ID -> Hotkey -> Object with min_version axon_version and extrinsics_version
result = {}

for index, subnet in enumerate(subnets_range):
    try:
        extrinsics = aggregated_extrinsics[subnet]
        axons = axon_infos[index]

        result[subnet] = {}
        
        for axon in axons:
            hotkey = axon.hotkey
            version = axon.version

            if hotkey in extrinsics:
                extrinsic = extrinsics[hotkey]
                extrinsic_version = extrinsic.version
            else:
                extrinsic_version = None

            result[subnet][hotkey] = {
                "axon_version": version,
                "subm_version": extrinsic_version,
                "transaction": extrinsic.txHash if extrinsic_version is not None else None
            }
    except Exception as e:
        print(f"Error at subnet {subnet}: {e}")

result

In [18]:
# Convert to json
import json

json_result = json.dumps(result, indent=4)

# Save json
with open("result.json", "w") as file:
    file.write(json_result)